# Install Testing Notebook
### Purpose of this file is to test things like FLAN-T5 downloads.

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

In [4]:
import torch
torch.cuda.is_available()

True

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

/opt/conda/envs/langgenv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/envs/langgenv2/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) 

<pad> Wie old sind Sie?</s>


In [56]:
input_text = "Encipher the following text using a caesar cipher with right shift of 3: hello my name is austin"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> hm i am austin</s>


In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [1]:
import re

In [2]:
char_to_num = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13,
    'o': 14,
    'p': 15,
    'q': 16,
    'r': 17,
    's': 18,
    't': 19,
    'u': 20,
    'v': 21,
    'w': 22,
    'x': 23,
    'y': 24,
    'z': 25,
}


def format_text(text):
    plaintext = re.sub(r'[^A-Za-z ]+', '', text)  # Remove all non alphabet text except spaces
    plaintext = re.sub(' +', ' ', plaintext)  # Remove extra spaces
    return plaintext.lower()


# NOTE: shift can be negative (left) or positive (right)
# If encode=True, encipher text, otherwise decipher
def caesar_cipher(original, shift, encode):
    if encode:
        myshift = shift
    else:
        myshift = shift * -1
    newtext = ''
    for i in original:
        if i == ' ':  # Preserve spaces
            newtext += ' '
        else:
            newnum = (char_to_num[i] + myshift) % 26
            newchar = list(char_to_num.keys())[list(char_to_num.values()).index(newnum)]
            newtext += newchar
    return newtext

In [66]:
mytext = format_text("Hello my name is 6786876 Austin!")
print(f"Original Text: '{mytext}'")
shift = 25
ctext = caesar_cipher(mytext, shift, encode=True)
print(f"Ciphered Text: '{ctext}'")
dtext = caesar_cipher(ctext, shift, encode=False)
print(f"Deciphered Text: '{dtext}'")

Original Text: 'hello my name is austin'
Ciphered Text: 'gdkkn lx mzld hr ztrshm'
Deciphered Text: 'hello my name is austin'


In [4]:
mytext = format_text("May the Force be with you.")
print(f"Original Text: '{mytext}'")
shift = -87
ctext = caesar_cipher(mytext, shift, encode=True)
print(f"Ciphered Text: '{ctext}'")
dtext = caesar_cipher(ctext, shift, encode=False)
print(f"Deciphered Text: '{dtext}'")

Original Text: 'may the force be with you'
Ciphered Text: 'drp kyv wfitv sv nzky pfl'
Deciphered Text: 'may the force be with you'
